<a href="https://colab.research.google.com/github/thaisgulias/DataAnalytics_student/blob/main/Exercicio_09_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalações

In [ ]:
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from google.cloud import storage
import os

##Cloud-ocorrencias

In [ ]:
#CONFIGURANDO DA CHAVE DE SEGURANCA (Enviada com o projeto)

serviceAccount = '/content/compact-epigram-377819-06a43c21ad20.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#Configurações Google Cloud Storage
client = storage.Client()
bucket = client.get_bucket('aulapandas')
bucket.blob('ocorrenciass.csv')
path = 'gs://aulapandas/brutos/ocorrenciass.csv'

In [ ]:
df = pd.read_csv(path,sep=';',encoding='ISO-8859-1',parse_dates=['ocorrencia_dia'],dayfirst=True)

##Subir sem tratar no mongo

In [ ]:
# selecionar o banco de dados e a coleção
db = client['cenipa']
colecao = db['naotratado']
colecao.estimated_document_count({})


# converter o DataFrame em uma lista de dicionários
documentos = df.to_dict(orient='records')

# inserir os documentos na coleção
colecao.insert_many(documentos)

##Ocorrencias-tratado

In [ ]:
# Drop de colunas identicas ( codigo_ocorrencia1 , codigo_ocorrencia2,codigo_ocorrencia3 , codigo_ocorrencia4)
df.drop(['codigo_ocorrencia1','codigo_ocorrencia2','codigo_ocorrencia3','codigo_ocorrencia4'],axis=1,inplace=True)
# Drop de colunas com problemas e/ou não utilizadas na analise
df.drop(['ocorrencia_latitude','ocorrencia_longitude','investigacao_aeronave_liberada','investigacao_status','divulgacao_relatorio_numero','divulgacao_relatorio_publicado','divulgacao_dia_publicacao'],axis=1,inplace=True)

In [ ]:
#Renomeando colunas
df.rename(columns={'ocorrencia_classificacao':'classificacao','ocorrencia_cidade':'cidade','ocorrencia_uf':'uf','ocorrencia_pais':'pais'},inplace=True)
df.rename(columns={'ocorrencia_aerodromo':'aerodromo','ocorrencia_dia':'data','ocorrencia_hora':'horario','total_aeronaves_envolvidas':'n_aeronaves','ocorrencia_saida_pista':'saida_pista'},inplace=True)

In [ ]:
df.replace(['***','**NI'],pd.NA,inplace=True)



In [ ]:
df.replace(pd.NA,np.NaN,inplace=True)

In [ ]:
df.shape

(6769, 11)

##mongo

In [ ]:
pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pymongo
from pymongo import MongoClient

In [ ]:
#Concector MongoDB Atlas
uri = "mongodb+srv://cluster0.jjkb9co.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,tls=True,tlsCertificateKeyFile='/content/X509-cert-3926889476236238374.pem')
                     
db = client['soulcode']
colecao = db['aulamongo']

##Enviar o DF-ocorrencias tratado

In [ ]:
db2 = client['cenipa']
colecaocenipa = db2['tratado']
colecaocenipa.count_documents({})

#ocorrendo erro de sintaxe foi necessario rodar novamento o inicio do notebook para validar o mongo

0

In [ ]:
df_dict = df.to_dict("records")

colecaocenipa.insert_many(df_dict)

In [ ]:
colecaocenipa.count_documents({})

6769

In [ ]:
#Extrair de uma colecao do MongoDB
cursor = colecaocenipa.find({})
dfcenipa = pd.DataFrame(list(cursor))

In [ ]:
dfcenipa

,_id,codigo_ocorrencia,classificacao,cidade,uf,pais,aerodromo,data,horario,total_recomendacoes,n_aeronaves,saida_pista
0,640a39a255be63e44cc092b3,81027,INCIDENTE GRAVE,SÃO PAULO,SP,BRASIL,SBSP,2022-10-09,16:30:00,0,1,SIM
1,640a39a255be63e44cc092b4,81030,INCIDENTE,VITÓRIA,ES,BRASIL,SBVT,2022-10-09,16:20:00,0,1,NÃO
2,640a39a255be63e44cc092b5,81023,INCIDENTE GRAVE,AMERICANA,SP,BRASIL,SDAI,2022-10-06,15:30:00,0,1,SIM
3,640a39a255be63e44cc092b6,81029,INCIDENTE,BELO HORIZONTE,MG,BRASIL,SBBH,2022-10-06,00:44:00,0,1,NÃO
4,640a39a255be63e44cc092b7,81025,INCIDENTE,MANAUS,AM,BRASIL,SBEG,2022-10-05,20:56:00,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...
6764,640a39a255be63e44cc0ad1f,40324,INCIDENTE,PELOTAS,RS,BRASIL,SBPK,2010-01-05,19:25:00,0,1,NÃO
6765,640a39a255be63e44cc0ad20,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,BRASIL,NaN,2010-01-04,17:30:00,0,1,NÃO
6766,640a39a255be63e44cc0ad21,40211,INCIDENTE,RIO DE JANEIRO,RJ,BRASIL,NaN,2010-01-03,12:00:00,0,1,NÃO
6767,640a39a255be63e44cc0ad22,40349,INCIDENTE,BELÉM,PA,BRASIL,SBBE,2010-01-03,11:05:00,0,1,NÃO


##Cloud-Airbnb

In [ ]:
#CONFIGURANDO DA CHAVE DE SEGURANCA (Enviada com o projeto)

serviceAccount = '/content/compact-epigram-377819-06a43c21ad20.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#Configurações Google Cloud Storage
client = storage.Client()
bucket = client.get_bucket('aulapandas')
bucket.blob('amsterdam_weekdays.csv')
path = 'gs://aulapandas/brutos/amsterdam_weekdays.csv'

In [ ]:
df = pd.read_csv(path,sep=',')

In [ ]:
df

,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,0,194.033698,Private room,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772
1,1,344.245776,Private room,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432
2,2,264.101422,Private room,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103
3,3,433.529398,Private room,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663
4,4,485.552926,Private room,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,1098,2486.115342,Entire home/apt,False,False,2.0,False,0,0,10.0,100.0,1,2.375833,1.436054,181.198355,9.594573,225.304203,15.699521,4.85869,52.37677
1099,1099,233.637194,Private room,False,True,2.0,False,1,0,10.0,90.0,1,4.719736,0.322263,108.024880,5.719989,134.775277,9.391335,4.83611,52.34910
1100,1100,317.062311,Private room,False,True,2.0,False,1,0,10.0,92.0,1,0.598355,0.751993,440.476515,23.323524,625.947562,43.616927,4.88897,52.37798
1101,1101,1812.855904,Entire home/apt,False,False,4.0,False,0,0,8.0,84.0,5,1.943020,0.388532,257.948105,13.658524,336.589237,23.454022,4.90688,52.35794


##Subir sem tratar no mongo

In [ ]:
# selecionar o banco de dados e a coleção
db = client['airbnba']
colecao = db['naotratado']
colecao.estimated_document_count({})


# converter o DataFrame em uma lista de dicionários
documentos = df.to_dict(orient='records')

# inserir os documentos na coleção
colecao.insert_many(documentos)

##airbnba tratado


In [ ]:
df = df.rename(columns={
    'realSum': 'preço total',
    'room_type': 'tipo de quarto',
    'room_shared': 'quarto compartilhado',
    'room_private': 'quarto privativo',
    'person_capacity': 'capacidade de hospedes',
    'host_is_superhost': 'superhost',
    'multi': 'n quartos',
    'biz': 'tipo',
    'cleanliness_rating': 'Classificação de limpeza',
    'guest_satisfaction_overall': 'satisfação',
    'bedrooms': 'número de quartos'
})
#traduzindo nome das colunas 

In [ ]:
df['tipo de quarto'].replace({'Private room': 'privado', 'Entire home/apt': 'hosp_inteiro', 'Shared room': 'compartilhado'}, inplace=True) #traduzindo as variaveis de room_type


In [ ]:
colunas_para_remover = ['attr_index', 'attr_index_norm', 'rest_index', 'rest_index_norm']
df = df.drop(columns=colunas_para_remover) # removendo as colunas indesejadas

In [ ]:
df

,Unnamed: 0,preço total,tipo de quarto,quarto compartilhado,quarto privativo,capacidade de hospedes,superhost,n quartos,tipo,Classificação de limpeza,satisfação,número de quartos,dist,metro_dist,lng,lat
0,0,194.033698,privado,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,4.90569,52.41772
1,1,344.245776,privado,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,4.90005,52.37432
2,2,264.101422,privado,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,4.97512,52.36103
3,3,433.529398,privado,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,4.89417,52.37663
4,4,485.552926,privado,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,4.90051,52.37508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,1098,2486.115342,hosp_inteiro,False,False,2.0,False,0,0,10.0,100.0,1,2.375833,1.436054,4.85869,52.37677
1099,1099,233.637194,privado,False,True,2.0,False,1,0,10.0,90.0,1,4.719736,0.322263,4.83611,52.34910
1100,1100,317.062311,privado,False,True,2.0,False,1,0,10.0,92.0,1,0.598355,0.751993,4.88897,52.37798
1101,1101,1812.855904,hosp_inteiro,False,False,4.0,False,0,0,8.0,84.0,5,1.943020,0.388532,4.90688,52.35794


##Enviar o DF-airbnba tratado

In [ ]:
db2 = client['airbnba']
colecao = db2['tratado']
colecao.count_documents({})

#ocorrendo erro de sintaxe foi necessario rodar novamento o inicio do notebook para validar o mongo

0

In [ ]:
df_dict = df.to_dict("records")

colecao.insert_many(df_dict)

In [ ]:
colecao.count_documents({})

1103

In [ ]:
#Extrair de uma colecao do MongoDB
cursor = colecao.find({})
dfairbnba = pd.DataFrame(list(cursor))

In [ ]:
dfairbnba

,_id,Unnamed: 0,preço total,tipo de quarto,quarto compartilhado,quarto privativo,capacidade de hospedes,superhost,n quartos,tipo,Classificação de limpeza,satisfação,número de quartos,dist,metro_dist,lng,lat
0,640a4a0f55be63e44cc0dd28,0,194.033698,privado,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,4.90569,52.41772
1,640a4a0f55be63e44cc0dd29,1,344.245776,privado,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,4.90005,52.37432
2,640a4a0f55be63e44cc0dd2a,2,264.101422,privado,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,4.97512,52.36103
3,640a4a0f55be63e44cc0dd2b,3,433.529398,privado,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,4.89417,52.37663
4,640a4a0f55be63e44cc0dd2c,4,485.552926,privado,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,4.90051,52.37508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,640a4a0f55be63e44cc0e172,1098,2486.115342,hosp_inteiro,False,False,2.0,False,0,0,10.0,100.0,1,2.375833,1.436054,4.85869,52.37677
1099,640a4a0f55be63e44cc0e173,1099,233.637194,privado,False,True,2.0,False,1,0,10.0,90.0,1,4.719736,0.322263,4.83611,52.34910
1100,640a4a0f55be63e44cc0e174,1100,317.062311,privado,False,True,2.0,False,1,0,10.0,92.0,1,0.598355,0.751993,4.88897,52.37798
1101,640a4a0f55be63e44cc0e175,1101,1812.855904,hosp_inteiro,False,False,4.0,False,0,0,8.0,84.0,5,1.943020,0.388532,4.90688,52.35794


##Cloud-Deputados

In [ ]:
#CONFIGURANDO DA CHAVE DE SEGURANCA (Enviada com o projeto)

serviceAccount = '/content/compact-epigram-377819-06a43c21ad20.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#Configurações Google Cloud Storage
client = storage.Client()
bucket = client.get_bucket('aulapandas')
bucket.blob('depsp.csv')
path = 'gs://aulapandas/brutos/depsp.csv'

In [ ]:
df = pd.read_csv(path,sep=',')

In [ ]:
df

,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
...,...,...,...,...,...,...,...,...
542390,2007,300441,3,120.00,6306675000140.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",OPTEI EXPRESS LTDA.
542391,2007,300441,3,2096.80,52204617000122.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHURRASCARIA JANDA LTDA.
542392,2007,300441,3,13.48,62925474000282.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",ASSOCIAÇÃO FUNC.ASSEMB.LEG. EST.SÃO PAULO
542393,2007,300441,3,38.08,66747577000170.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",IBIZA EUROPEIA ALIMENTOS LTDA


##Subir sem tratar no mongo

In [ ]:
# selecionar o banco de dados e a coleção
db = client['sp']
colecaosp = db['naotratado']
colecaosp.estimated_document_count({})


# converter o DataFrame em uma lista de dicionários
documentos = df.to_dict(orient='records')

# inserir os documentos na coleção
colecaosp.insert_many(documentos)

##Deputados-Tratados

In [ ]:
df.isna()

,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
542390,False,False,False,False,False,False,False,False
542391,False,False,False,False,False,False,False,False
542392,False,False,False,False,False,False,False,False
542393,False,False,False,False,False,False,False,False


In [ ]:
df.isna().sum()

Ano               0
Matricula         0
Mes               0
Valor             0
CNPJ          10967
Deputado          0
Tipo              0
Fornecedor        0
dtype: int64

In [ ]:
df_x = df.copy()

In [ ]:
df_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542395 entries, 0 to 542394
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Ano         542395 non-null  int64  
 1   Matricula   542395 non-null  int64  
 2   Mes         542395 non-null  int64  
 3   Valor       542395 non-null  float64
 4   CNPJ        531428 non-null  object 
 5   Deputado    542395 non-null  object 
 6   Tipo        542395 non-null  object 
 7   Fornecedor  542395 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 33.1+ MB


In [ ]:
df_x.isnull().sum()

Ano               0
Matricula         0
Mes               0
Valor             0
CNPJ          10967
Deputado          0
Tipo              0
Fornecedor        0
dtype: int64

In [ ]:
df_x.duplicated().sum()

0

In [ ]:
# Remover linhas duplicadas
#df_x.drop_duplicates()

In [ ]:
df_x.fillna(0)

,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
...,...,...,...,...,...,...,...,...
542390,2007,300441,3,120.00,6306675000140.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",OPTEI EXPRESS LTDA.
542391,2007,300441,3,2096.80,52204617000122.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHURRASCARIA JANDA LTDA.
542392,2007,300441,3,13.48,62925474000282.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",ASSOCIAÇÃO FUNC.ASSEMB.LEG. EST.SÃO PAULO
542393,2007,300441,3,38.08,66747577000170.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",IBIZA EUROPEIA ALIMENTOS LTDA


In [ ]:
#df_x['Valor'] = df_x['Valor'].astype(int)

In [ ]:
df_x.CNPJ = df_x.CNPJ.replace('O','0')

In [ ]:
correcoes = { 
    'VIVO':'VIVO S.A.', 
    'TELEFONICA BRASIL S.A':'TELEFONICA BRASIL S.A.', 
    'NEXTEL TELECOMUNICAÇÕES LTDA':'NEXTEL TELECOMUNICAÇÕES LTDA.', 
    'SABESP':'COMPANHIA DE SANEAMENTO BÁSICO DO ESTADO DE SÃO PAULO - SABESP',
    'TELEFÔNICA BRASIL S.A.':'TELEFONICA BRASIL S.A.',
    'TELEFONICA': 'TELEFONICA BRASIL S.A.',
    'COMPANHIA PAULISTA DE FORÇA E LUZ':'CPFL ENERGIA', 
    'TELEFONICA BRASIL S/A':'TELEFONICA BRASIL S.A.',
    'TELEFONICA BRASIL SA':'TELEFONICA BRASIL S.A.',
    'CLARO S.A.':'CLARO S/A',
    'ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO PAULO S.A.':'ELETROPAULO',
    'ELETROPAULO':'AES ELETROPAULO',
    'CLARO S.A':'CLARO S/A',
    'NEXTEL TELECOMUNICAÇÕES LTDA.':'NEXTEL TELECOMUNICAÇÕES LTDA', 
    'NET':'NET SERVIÇOS DE COMUNICAÇÃO S/A',
    'TELEFÔNICA BRASIL S.A':'TELEFONICA BRASIL S.A.',  
    'KALUNGA':'KALUNGA COM. E IND. GRÁFICA LTDA.',
    'KALUNGA COMERCIO E IND. GRAFICA LTDA':'KALUNGA COM. E IND. GRÁFICA LTDA.',
    'NEXTEL':'NEXTEL TELECOMUNICAÇÕES LTDA',
    'KALUNGA COM. E IND. GRAFICA LTDA':'KALUNGA COM. E IND. GRÁFICA LTDA.', 
    'TELEFÔNICA BRASIL S/A':'TELEFONICA BRASIL S.A.',
    'TELESP CELULAR':'TELESP CELULAR S/A'
}
  
for key, value in correcoes.items():
  df_x.loc[df.Fornecedor==key,'Fornecedor'] = value

In [ ]:
df_x

,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,2015,300257,3,200,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,2015,300257,3,295,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,2015,300257,3,2850,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,2015,300257,4,100,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,2015,300257,4,195,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
...,...,...,...,...,...,...,...,...
542390,2007,300441,3,120,6306675000140.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",OPTEI EXPRESS LTDA.
542391,2007,300441,3,2096,52204617000122.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHURRASCARIA JANDA LTDA.
542392,2007,300441,3,13,62925474000282.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",ASSOCIAÇÃO FUNC.ASSEMB.LEG. EST.SÃO PAULO
542393,2007,300441,3,38,66747577000170.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",IBIZA EUROPEIA ALIMENTOS LTDA


##Enviar o DF-deputados tratado

In [ ]:
db2 = client['sp']
colecaosp = db2['tratado']
colecaosp.count_documents({})

#ocorrendo erro de sintaxe foi necessario rodar novamento o inicio do notebook para validar o mongo

0

In [ ]:
df_dict = df.to_dict("records")

colecaosp.insert_many(df_dict)

In [ ]:
colecaosp.count_documents({})

542395

In [ ]:
#Extrair de uma colecao do MongoDB
cursor = colecaosp.find({})
dfsp = pd.DataFrame(list(cursor))

In [ ]:
dfsp

,_id,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,640a4f4055be63e44cc92833,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,640a4f4055be63e44cc92834,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,640a4f4055be63e44cc92835,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,640a4f4055be63e44cc92836,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,640a4f4055be63e44cc92837,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
...,...,...,...,...,...,...,...,...,...
542390,640a4f4555be63e44cd16ee9,2007,300441,3,120.00,6306675000140.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",OPTEI EXPRESS LTDA.
542391,640a4f4555be63e44cd16eea,2007,300441,3,2096.80,52204617000122.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHURRASCARIA JANDA LTDA.
542392,640a4f4555be63e44cd16eeb,2007,300441,3,13.48,62925474000282.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",ASSOCIAÇÃO FUNC.ASSEMB.LEG. EST.SÃO PAULO
542393,640a4f4555be63e44cd16eec,2007,300441,3,38.08,66747577000170.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",IBIZA EUROPEIA ALIMENTOS LTDA
